In [ ]:
import pandas as pd
from pathlib import Path
from pandas.api.types import is_datetime64_any_dtype as is_datetime
import numpy as np
import src.utils as utils
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.pylabtools import figsize
import src.data_processing.data as data
from shapely.geometry import Point

In [ ]:
#Get total number of facilities per station:
facil_col = ['ticket_vending_machine', 'luggage_lockers', 'free_parking', 'taxi', 'bicycle_spots', 'blue-bike',
             'bus', 'tram', 'metro', 'wheelchair_available', 'ramp', 'disabled_parking_spots', 'elevated_platform',
             'escalator_up', 'escalator_down', 'elevator_platform', 'audio_induction_loop']
# PROBLEM: disabled_parking_spots is the number of spots instead of a dummy indicator
# so let's create a dummy variable
data.facilities['disabled_parking_spots_indicator'] = np.where(data.facilities['disabled_parking_spots']==0,0,1)
# drop old variable and add new name to our list
facil_col.remove('disabled_parking_spots')
facil_col = facil_col + ['disabled_parking_spots_indicator']
# compute total number of facilities per station
data.facilities['number_facilities'] = data.facilities[facil_col].sum(axis = 1)
data.facilities['number_facilities']

In [ ]:
## Number of daily travelers
data.travelers["sum"] = data.travelers["week"] + data.travelers["saturday"] + data.travelers["sunday"]
len(list(set(data.facilities['name']).intersection(set(data.travelers['Station']))))
intersection = list(set(data.facilities['name']).intersection(set(data.travelers['Station'])))
len(intersection)
still_needed = set(data.travelers['Station']).difference(intersection)
still_needed
merge = pd.merge(data.facilities, data.travelers, left_on='name', right_on='Station')
if(merge['sum'].empty != True):
    merge['daily_travelers'] = merge['sum']/7
merge

In [ ]:
#univariate regression for daily travelers:
import statsmodels.api as sm
# run multivariate regression
X = merge['daily_travelers']
Y = merge['number_facilities']
X = sm.add_constant(X) # adding a constant: Y = beta0 + beta1*X1 + beta2*X2 + espilon instead of Y = beta1*X1 + beta2*X2 + epsilon

model = sm.OLS(Y, X).fit()
print_model = model.summary()
print_model